In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
from google.colab import files
df = files.upload()

Saving 1377884570_tweet_global_warming.csv to 1377884570_tweet_global_warming.csv


In [ ]:
df = pd.read_csv('/content/1377884570_tweet_global_warming.csv', encoding='latin-1', on_bad_lines='skip', lineterminator='\r')

In [ ]:
df.drop(columns = 'existence.confidence', inplace = True)

In [ ]:
df

,tweet,existence
0,Global warming report urges governments to act...,Yes
1,Fighting poverty and global warming in Africa ...,Yes
2,Carbon offsets: How a Vatican forest failed to...,Yes
3,Carbon offsets: How a Vatican forest failed to...,Yes
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes
...,...,...
6085,"@bloodless_coup ""The phrase 'global warming' s...",Y
6086,Virginia to Investigate Global Warming Scienti...,NaN
6087,Global warming you tube parody you will enjoy ...,N
6088,One-Eyed Golfer: Don't dare tell me about glob...,N


In [ ]:
df['existence'].value_counts()

Y      2554
N      1053
Yes     557
No       61
Name: existence, dtype: int64

In [ ]:
df['existence'].isnull().sum()

1865

In [ ]:

df.dropna(inplace = True)

In [ ]:
df

,tweet,existence
0,Global warming report urges governments to act...,Yes
1,Fighting poverty and global warming in Africa ...,Yes
2,Carbon offsets: How a Vatican forest failed to...,Yes
3,Carbon offsets: How a Vatican forest failed to...,Yes
4,URUGUAY: Tools Needed for Those Most Vulnerabl...,Yes
...,...,...
6084,It's 83_Á and climbing in NYC. August weather...,Y
6085,"@bloodless_coup ""The phrase 'global warming' s...",Y
6087,Global warming you tube parody you will enjoy ...,N
6088,One-Eyed Golfer: Don't dare tell me about glob...,N


In [ ]:
df.loc[df['existence'] == 'Yes', 'existence'] = 'Y'
df.loc[df['existence'] == 'No', 'existence'] = 'N'

In [ ]:
df['existence'].value_counts()

Y    3111
N    1114
Name: existence, dtype: int64

In [ ]:
df['tweet'] = df['tweet'].str.replace('[#,@,&,:,;,_,?,|]','', regex = True)
df['tweet'] = df['tweet'].str.replace(r'link','', regex = True)
df['tweet'] = df['tweet'].str.replace(r'[..[]|[]]','', regex = True)
df['tweet'] = df['tweet'].str.replace('\(.*?\)','', regex = True)

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
!pip install contractions

In [ ]:
import contractions

for i in range(len(df)):
  df['tweet'][i] = contractions.fix(df['tweet'][i])
  df['tweet'][i] = df['tweet'][i].lower()

In [ ]:
df

,tweet,existence
0,global warming report urges governments to act...,Y
1,fighting poverty and global warming in africa,Y
2,carbon offsets how a vatican forest failed to ...,Y
3,carbon offsets how a vatican forest failed to ...,Y
4,uruguay tools needed for those most vulnerable...,Y
...,...,...
4220,it is 83á and climbing in nyc august weather ...,Y
4221,"bloodlesscoup ""the phrase 'global warming' sho...",Y
4222,global warming you tube parody you will enjoy ...,N
4223,one-eyed golfer do not dare tell me about glob...,N


In [ ]:
df['tweet'] = df['tweet'].str.replace('[-,á,à]','', regex = True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['existence'] = le.fit_transform(df['existence'])

In [ ]:
df

,tweet,existence
0,global warming report urges governments to act...,1
1,fighting poverty and global warming in africa,1
2,carbon offsets how a vatican forest failed to ...,1
3,carbon offsets how a vatican forest failed to ...,1
4,uruguay tools needed for those most vulnerable...,1
...,...,...
4220,it is 83 and climbing in nyc august weather f...,1
4221,"bloodlesscoup ""the phrase 'global warming' sho...",1
4222,global warming you tube parody you will enjoy ...,0
4223,oneeyed golfer do not dare tell me about globa...,0


In [ ]:
from sklearn.model_selection import train_test_split

x = df.iloc[:,0].values
y = df.iloc[:,1].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42) 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None) 

In [ ]:
round(sum([len(i.split()) for i in x_train])/len(x_train))

15

In [ ]:
max_vocab_length = 10000 
max_length = 15 

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(x_train)

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=max_length,
                             name="embedding_1") 

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") 
x = text_vectorizer(inputs)
x = embedding(x) 
x = layers.GlobalAveragePooling1D()(x) 
outputs = layers.Dense(1, activation="sigmoid")(x) 
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") 

In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1_history = model_1.fit(x_train,
                              y_train,
                              epochs=30, verbose = 0)

In [ ]:
model_1.evaluate(x_test, y_test)

14/14 [==============================] - 0s 7ms/step - loss: 0.5233 - accuracy: 0.8440


[0.5232744216918945, 0.8439716100692749]

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
model_1_pred_probs = model_1.predict(x_test)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) 
model_1_results = calculate_results(y_true=y_test, 
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 84.39716312056737,
 'f1': 0.8474008006109058,
 'precision': 0.852911486203315,
 'recall': 0.8439716312056738}